In [ ]:

!pip install fal_client fastapi python-dotenv uvicorn gradio openai

In [ ]:

import os

def save_secrets(api_key, api_base, fal_key):
    os.environ["API_KEY"] = api_key
    os.environ["API_BASE"] = api_base
    os.environ["FAL_KEY"] = fal_key


save_secrets(
    api_key="sk-or-v1-",   ### use openrouter api key
    api_base="https://openrouter.ai/api/v1",
    fal_key="1a83aad3-18"
)

print(os.environ.get("API_KEY"))
print(os.environ.get("API_BASE"))
print(os.environ.get("FAL_KEY"))

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
https://openrouter.ai/api/v1
1a83aad3-1801-4b6f-ac1d-14a3c82d95f5:d2f493d1878b0859c2233d19a3701fd8


In [ ]:
import subprocess
libraries = ['fal_client', 'fastapi', 'python-dotenv', 'uvicorn', 'gradio', 'openai']

for lib in libraries:
    result = subprocess.run(['pip', 'show', lib], capture_output=True, text=True)
    version = [line for line in result.stdout.split('\n') if 'Version:' in line][0]
    print(f"{lib}: {version}")

fal_client: Version: 0.5.6
fastapi: Version: 0.115.6
python-dotenv: Version: 1.0.1
uvicorn: Version: 0.34.0
gradio: Version: 5.9.1
openai: Version: 1.57.4


In [ ]:



import gradio as gr
import json
import re
import random
from openai import OpenAI
import fal_client
from pydantic import BaseModel
import os



api_key = os.getenv('API_KEY')
api_base = os.getenv('API_BASE')
FAL_KEY = os.getenv('FAL_KEY')

# Initialize OpenAI client
client = OpenAI(
    api_key=api_key,
    base_url=api_base
)

model = "google/gemini-flash-1.5"

class MemeRequest(BaseModel):
    domain: str

class MemeResponse(BaseModel):
    image_url: str
    top_text: str
    bottom_text: str

def generate_meme(domain: str) -> MemeResponse:
    """
    Generates a meme based on the given domain.
    This function is adapted from your FastAPI code.
    """
    temperature = round(random.uniform(0.5, 0.9), 2)
    print(f"Temperature: {temperature}")

    user_content = (f"Act like Non offensive meme maker. Always create different and unique funny memes "
                    f"always remember stable diffusion cannot render text natively hence , write prompt just detailing the scene or image without text"
                    f"Return meme details in below json format for topic {domain}\n\n"
                    "{ \"stableDiffusionPrompt\": \" \", \"topText\": \"\", \"bottomText\": \"\" }\n\n"
                    "Strictly just reply json no extra text")

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful meme maker, who makes non offensive memes(Dont include Cat in memes)"},
            {"role": "user", "content": user_content}
        ],
        model=model,
        temperature=temperature
    )

    result = chat_completion.choices[0].message.content

    try:
        meme_data = json.loads(result)
    except json.JSONDecodeError:
        json_match = re.search(r'```json\s*(.*?)\s*```', result, re.DOTALL)
        if json_match:
            json_string = json_match.group(1)
            meme_data = json.loads(json_string)
        else:
            raise ValueError("No valid JSON found in the response")

    print("Stable Diffusion Prompt:", meme_data['stableDiffusionPrompt'])
    print("Top Text:", meme_data['topText'])
    print("Bottom Text:", meme_data['bottomText'])



    handler = fal_client.submit(
        "fal-ai/flux/schnell",
        arguments={
            "prompt": meme_data['stableDiffusionPrompt'],
            "image_size": "landscape_4_3",
            "num_inference_steps": 4,
            "num_images": 1,
            "enable_safety_checker": True
        }
    )

    result = handler.get()
    image_url = result['images'][0]['url']
    print("Generated Image URL:", image_url)

    return MemeResponse(
        top_text=meme_data['topText'],
        image_url=image_url,
        bottom_text=meme_data['bottomText']  )

def generate_meme_gradio(domain):
    """
    Wrapper function for Gradio interface.
    """
    try:
        meme_response = generate_meme(domain)
        print(meme_response.top_text, meme_response.image_url, meme_response.bottom_text)
        return meme_response.top_text, meme_response.image_url, meme_response.bottom_text
    except Exception as e:
        return None, str(e), ""


# # Create Gradio interface
import gradio as gr

# Example domains
example_domains = [
    'HR', 'Technology', 'AI', 'Machine Learning', 'Sales', 'Marketing',
    'Remote Work', 'Coffee', 'Monday', 'Deadlines', 'Office Pranks', 'Tech Support',
    'Social Media', 'Startup Life', 'Zoom Fails', 'Work-Life Balance', 'Coding',
    'Data Privacy', 'Cybersecurity', 'Cloud Computing', 'Blockchain', 'IoT',
    'Virtual Reality', 'Augmented Reality', 'Quantum Computing', '5G', 'AI Ethics'
]


demo = gr.Interface(
    fn=generate_meme_gradio,
    inputs=[
        gr.Textbox(
            label="Enter your meme topic",
            placeholder="Type a topic or choose from examples below",
            info="Try topics like 'AI', 'Remote Work', or 'Tech Support'"
        )
    ],
    outputs=[
        gr.Textbox(label="Top Text"),
        gr.Image(label="Generated Meme"),
        gr.Textbox(label="Bottom Text")
    ],
    theme=gr.themes.Ocean(),
    title="🎨 AI Meme Factory",
    description="""
    Welcome to the AI Meme Factory! World's First Generative meme Engine.
    Generate unique, funny, and non-offensive memes using the power of AI.

    🤖 Powered by:
    - Gemini for creative text generation
    - Flux for high-quality image generation

    ✨ Features:
    - Instant meme generation
    - Professional and work-appropriate content
    - Wide range of topics supported
    """,
    examples=[[domain] for domain in random.sample(example_domains, 6)],
    allow_flagging="never",
    cache_examples=False
)
demo.launch(  share=True,   debug=True )




Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4e6465b6f4a9aa9b9e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Temperature: 0.68
Stable Diffusion Prompt: A person wearing AR glasses, superimposed holographic images of tools and instructions floating around them as they assemble a complex piece of furniture in a living room. The furniture is mostly assembled, showing progress.
Top Text: Me trying to follow IKEA instructions
Bottom Text: With Augmented Reality
Generated Image URL: https://fal.media/files/penguin/IUJM8p_POzBXftRG1M3Xs.png
Me trying to follow IKEA instructions https://fal.media/files/penguin/IUJM8p_POzBXftRG1M3Xs.png With Augmented Reality
